# Making an image of the Mandelbrot set using `htcondor-dags`

## The shape of the workflow

In [ ]:
from graphviz import Digraph
import itertools

num_tiles_per_side = 2

dot = Digraph()

dot.node('montage')
for x, y in itertools.product(range(num_tiles_per_side), repeat = 2):
    n = f'tile_{x}-{y}'
    dot.node(n)
    dot.edge(n, 'montage')

dot

## Running `goatbrot` locally

`goatbrot` options:
- `-i 1000` The number of iterations.
- `-o test.ppm` The name of the output file to generate.
- `-c 0,0` The center point of the image region.
- `-w 3` The width of the image region.
- `-s 1000,1000` The pixel dimensions of the final image.

In [ ]:
! ./goatbrot -i 1000 -c 0,0 -w 3 -s 1000,1000 -o test.ppm
! convert test.ppm test.png

## Describing `goatbrot` as an HTCondor job

In [ ]:
import htcondor

tile_description = htcondor.Submit(
    executable = 'goatbrot',
    arguments = '-i 1000 -c $(x),$(y) -w $(w) -s 500,500 -o tile_$(tile_x)-$(tile_y).ppm',
    log = 'mandelbrot.log',
    output = 'goatbrot.out.$(tile_x)_$(tile_y)',
    error = 'goatbrot.err.$(tile_x)_$(tile_y)',
    request_cpus = '1',
    request_memory = '128MB',
    request_disk = '1GB',
)

print(tile_description)

In [ ]:
def make_tile_vars(num_tiles_per_side, width = 3):
    width_per_tile = width / num_tiles_per_side
    
    centers = [
        width_per_tile * (n + 0.5 - (num_tiles_per_side / 2)) 
        for n in range(num_tiles_per_side)
    ]
    
    vars = []
    for (tile_y, y), (tile_x, x) in itertools.product(enumerate(centers), repeat = 2):
        var = dict(
            w = width_per_tile,
            x = x,
            y = -y,
            tile_x = str(tile_x).rjust(5, '0'),
            tile_y = str(tile_y).rjust(5, '0'),
        )
        
        vars.append(var)
        
    return vars

In [ ]:
tile_vars = make_tile_vars(2)
for var in tile_vars:
    print(var)

## Describing montage as an HTCondor job

In [ ]:
def make_montage_description(tile_vars):
    num_tiles_per_side = int(len(tile_vars) ** .5)
    
    input_files = [f'tile_{d["tile_x"]}-{d["tile_y"]}.ppm' for d in tile_vars]
    
    return htcondor.Submit(
        executable = '/usr/bin/montage',
        arguments = f'{" ".join(input_files)} -mode Concatenate -tile {num_tiles_per_side}x{num_tiles_per_side} mandelbrot.png',
        transfer_input_files = ', '.join(input_files),
        log = 'mandelbrot.log',
        output = 'montage.out',
        error = 'montage.err',
        request_cpus = '1',
        request_memory = '128MB',
        request_disk = '1GB',
    )

In [ ]:
montage_description = make_montage_description(make_tile_vars(2))

print(montage_description)

## Describing the DAG using `htcondor-dags`

In [ ]:
import htcondor_dags as dags

num_tiles_per_side = 10

tile_vars = make_tile_vars(num_tiles_per_side)

dag = dags.DAG()

tile_layer = dag.layer(
    name = 'tile',
    submit_description = tile_description,
    vars = tile_vars,
)

montage_layer = tile_layer.child(
    name = 'montage',
    submit_description = make_montage_description(tile_vars),
)

In [ ]:
print(dag.describe())

## Write the DAG to disk

In [ ]:
from pathlib import Path
import shutil

dag_dir = Path('mandelbrot-dag').absolute()

shutil.rmtree(dag_dir)

dag_file = dag.write(dag_dir)
shutil.copy2('goatbrot', dag_dir)

print('DAG directory:', dag_dir)
print('DAG file:', dag_file)

## Submit the DAG via the bindings

In [ ]:
dag_submit = htcondor.Submit.from_dag(str(dag_file), {'force': 1})

print(dag_submit)

In [ ]:
import os
os.chdir(dag_dir)

schedd = htcondor.Schedd()
with schedd.transaction() as txn:
    cluster_id = dag_submit.queue(txn)
    
print(cluster_id)
os.chdir('..')

In [ ]:
! tail -f mandelbrot-dag/dagfile.dag.dagman.out